## Extract Poses from Amass Dataset

In [24]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import zipfile
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm



from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Please remember to download the following subdataset from AMASS website: https://amass.is.tue.mpg.de/download.php. Note only download the <u>SMPL+H G</u> data.
* ACCD (ACCD)
* HDM05 (MPI_HDM05)
* TCDHands (TCD_handMocap)
* SFU (SFU)
* BMLmovi (BMLmovi)
* CMU (CMU)
* Mosh (MPI_mosh)
* EKUT (EKUT)
* KIT  (KIT)
* Eyes_Janpan_Dataset (Eyes_Janpan_Dataset)
* BMLhandball (BMLhandball)
* Transitions (Transitions_mocap)
* PosePrior (MPI_Limits)
* HumanEva (HumanEva)
* SSM (SSM_synced)
* DFaust (DFaust_67)
* TotalCapture (TotalCapture)
* BMLrub (BioMotionLab_NTroje)

### Unzip all datasets. In the bracket we give the name of the unzipped file folder. Please correct yours to the given names if they are not the same.

### Place all files under the directory **./amass_data/**. The directory structure shoud look like the following:  
./amass_data/  
./amass_data/ACCAD/  
./amass_data/BioMotionLab_NTroje/  
./amass_data/BMLhandball/  
./amass_data/BMLmovi/   
./amass_data/CMU/  
./amass_data/DFaust_67/  
./amass_data/EKUT/  
./amass_data/Eyes_Japan_Dataset/  
./amass_data/HumanEva/  
./amass_data/KIT/  
./amass_data/MPI_HDM05/  
./amass_data/MPI_Limits/  
./amass_data/MPI_mosh/  
./amass_data/SFU/  
./amass_data/SSM_synced/  
./amass_data/TCD_handMocap/  
./amass_data/TotalCapture/  
./amass_data/Transitions_mocap/  

**Please make sure the file path are correct, otherwise it can not succeed.**

In [25]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
from human_body_prior.body_model.body_model import BodyModel

neutral_bm_path = '/home/shrik/HumanML3D/body_models/SMPLX_NEUTRAL_2020.npz'
num_betas = 16
neutral_bm = BodyModel(bm_fname=neutral_bm_path, num_betas=num_betas, num_expressions=0).to(comp_device)
faces = c2c(neutral_bm.f)

In [27]:
paths = []
folders = []
dataset_names = []
for root, dirs, files in os.walk('./amass_data'):
#     print(root, dirs, files)
#     for folder in dirs:
#         folders.append(os.path.join(root, folder))
    folders.append(root)
    for name in files:
        dataset_name = root.split('/')[2]
        if dataset_name not in dataset_names:
            dataset_names.append(dataset_name)
        if name.endswith('.npz'):
            paths.append(os.path.join(root, name))

In [28]:
save_root = './pose_data'
save_folders = [folder.replace('./amass_data', './pose_data') for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)
group_path = [[path for path in paths if name in path] for name in dataset_names]

In [29]:
trans_matrix = np.array([[1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0],
                         [0.0, 1.0, 0.0]])
ex_fps = 20
def amass_to_pose(src_path, save_path):
    bdata = np.load(src_path, allow_pickle=True)
    fps = 0
    try:
        fps = bdata['mocap_frame_rate']
    except:
        # for k,v in bdata.items():
        #     print(k,v.shape)
        # print(src_path)
        return fps
    if bdata['surface_model_type'].item() not in {'smplx','smplx_locked_head'}:
        return fps
    assert bdata['gender'] == 'neutral'
    bm = neutral_bm
    down_sample = int(fps / ex_fps)
#     print(frame_number)
#     print(fps)

    bdata_poses = bdata['poses'][::down_sample,...]
    bdata_trans = bdata['trans'][::down_sample,...]
    body_parms = {
            'root_orient': torch.Tensor(bdata_poses[:, :3]).to(comp_device),
            'pose_body': torch.Tensor(bdata_poses[:, 3:66]).to(comp_device),
            'pose_hand': torch.Tensor(bdata_poses[:, 75:]).to(comp_device),
            'trans': torch.Tensor(bdata_trans).to(comp_device),
            'betas': torch.Tensor(np.repeat(bdata['betas'][:num_betas][np.newaxis], repeats=len(bdata_trans), axis=0)).to(comp_device),
        }
    
    with torch.no_grad():
        body = bm(**body_parms)
    pose_seq_np = body.Jtr.detach().cpu().numpy()
    pose_seq_np_n = np.dot(pose_seq_np, trans_matrix)
    
    
    np.save(save_path, pose_seq_np_n)
    return fps

In [30]:
group_path = group_path
all_count = sum([len(paths) for paths in group_path])
cur_count = 0

In [31]:
import time
bad_zip_files = list()
unused_files = list()
for paths in group_path:
    dataset_name = paths[0].split('/')[2]
    fps = 0
    for path in tqdm(paths,desc='Processing: %s'%dataset_name,ncols=150):
        save_path = path.replace('./amass_data', './pose_data')
        save_path = save_path[:-3] + 'npy'
        if os.path.exists(save_path):
            continue
        try:
            fps = amass_to_pose(path, save_path)
        except zipfile.BadZipFile:
            bad_zip_files.append(path)
        except:
            unused_files.append(path)
    cur_count += len(paths)
    print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
    time.sleep(0.5)
for f in bad_zip_files:
    print('bad zip file:',f)
for f in unused_files:
    print('unused file:',f)

Processing: MoSh: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2049/2049 [00:02<00:00, 928.61it/s]


Processed / All (fps 0): 2049/19554


Processing: CMU: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 2079/2079 [00:11<00:00, 186.99it/s]


Processed / All (fps 0): 4128/19554


Processing: KIT: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 4287/4287 [00:19<00:00, 217.54it/s]


...

Processed / All (fps 0): 19377/19554


Processing: PosePrior: 100%|███████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 56299.38it/s]


Processed / All (fps 0): 19415/19554


Processing: DFaust: 100%|████████████████████████████████████████████████████████████████████████████████████████| 139/139 [00:00<00:00, 75626.96it/s]


Processed / All (fps 0): 19554/19554
bad zip file: ./amass_data/BMLmovi/Subject_49_F_MoSh/Subject_49_F_19_stageii.npz
bad zip file: ./amass_data/BMLmovi/Subject_49_F_MoSh/Subject_49_F_19_stageii.npz


## Segment, Mirror and Relocate Motions

In [32]:
from collections import defaultdict
import csv
import os
import pathlib

from tqdm import tqdm
import numpy as np

bm_params_f = pathlib.Path('./body_models/SMPLX_NEUTRAL_2020.npz')
index_f = pathlib.Path('./index.csv')
pose_data_d  = pathlib.Path('./pose_data')
joints_d = pathlib.Path('./joints')
fps = 20

Find the corresponding left/right joints from model npy file. We will mirror left/right joints to augment data.

In [33]:
bm_params = np.load(bm_params_f,allow_pickle=True)
joint2ind = bm_params['joint2num'].item()
ind2joint = {v:k
             for k,v in joint2ind.items()}
l_joints,r_joints = list(),list()
for j in joint2ind:
    if j.startswith('L_'):
        l_j = j
        r_j = j.replace('L_','R_')
        l_joints.append(joint2ind[l_j])
        r_joints.append(joint2ind[r_j])
# print('num joints to swap:',len(l_joints))
# for l,r in sorted(zip(l_joints,r_joints)):
#     print(ind2joint[l],ind2joint[r])
joints_to_drop = [joint2ind['Jaw'],
                  joint2ind['L_Eye'],
                  joint2ind['R_Eye']]

To sample frames according to HumanML3D, create the dictionary of files to sample, their start/end frames, their original ids.

In [34]:
to_sample = defaultdict(list)
n_dropped = 0
for row in csv.DictReader(open(index_f)):
    data_f = row['source_path']
    # Not using `humanact12` dataset. Discard those entries in val set.
    if 'humanact12' in data_f:
        n_dropped += 1
        continue
    # SMPL-X version of AMASS is missing the following. Ignore them.
    if ('BMLhandball' in data_f or
        'DanceDB' in data_f or
        'HUMAN4D' in data_f or 
        'CMU/22_23_Rory' in data_f or
        'CMU/18_19_rory' in data_f or
        'CMU/18_19_Justin' in data_f or
        'CMU/20_21_rory1' in data_f or
        'CMU/20_21_Justin1' in data_f or
        'CMU/22_23_justin' in data_f):
        n_dropped += 1
        continue
    # SMPL-X version of AMASS has renamed many files. Map the old names to new.
    data_f = (data_f
              .replace('/BioMotionLab_NTroje/','/BMLrub/')
              .replace('/DFaust_67/','/DFaust/')
              .replace('/MPI_mosh/','/MoSh/')
              .replace('/MPI_HDM05/','/HDM05/')
              .replace('/MPI_Limits/','/PosePrior/')
              .replace('/SSM_synced/','/SSM/')
              .replace('/TCD_handMocap/','/TCDHands/')
              .replace('/Transitions_mocap/','/Transitions/')
              .replace('.npz','')
              .replace('.npy','')
              .replace('_poses','')
              .replace(' ','_'))
    data_f = pathlib.Path(data_f)
    data_d = data_f.parent
    assert data_d.is_dir(),data_d
    for f in data_d.iterdir():
        if f.name.startswith(data_f.name):
            to_sample[f].append((int(row['start_frame']),
                                 int(row['end_frame']),
                                 row['new_name'].replace('.npy','')))
            break
    else:
        assert False

Many files need to be sampled multiple times because there are multiple entries in `index.csv` for those files.

In [35]:
print('samples dropped:',n_dropped)
print('files to sample:',len(to_sample))
print('files with >1 sample:', 
      sum(1
          for v in to_sample.values()
          if len(v) > 1))
print('total samples:',
      sum(len(v)
          for v in to_sample.values()))

samples dropped: 1365
files to sample: 10220
files with >1 sample: 2459
total samples: 13251


Gather all file names.

In [36]:
data_fs = list()
for rt,ds,fs in os.walk(pose_data_d):
    rt = pathlib.Path(rt)
    for f in fs:
        f = rt / f
        if f.suffix != '.npy':
            continue
        data_fs.append(f)
data_fs.sort()
print('num files:',len(data_fs))

num files: 17138


Sample each file as per HumanML3D, create a mirrored version of the sample, save both.

In [37]:
n_dropped = 0
pose_data_to_joints_map = ['file,new_id,orig_id']
pbar = tqdm(data_fs,desc='mirroring/pruning files',ncols=150)
for i,f in enumerate(pbar):
    if f not in to_sample:
        n_dropped += 1
        continue
    for j,(i_beg,i_end,orig_id) in enumerate(sorted(to_sample[f])):
        id = f'{i:06}_{j:02}'
        id_m = f'M{i:06}_{j:02}'
        out_f = joints_d / f'{id}.npy'
        out_m_f = joints_d / f'{id_m}.npy'
        data = np.load(f)
        if 'humanact12' not in str(f):
            if 'Eyes_Japan_Dataset' in str(f):
                data = data[3*fps:]
            if 'HDM05' in str(f):
                data = data[3*fps:]
            if 'TotalCapture' in str(f):
                data = data[1*fps:]
            if 'PosePrior' in str(f):
                data = data[1*fps:]
            if 'Transitions' in str(f):
                data = data[int(0.5*fps):]
        data = data[i_beg:i_end]
        if out_f.is_file() and out_m_f.is_file():
            pose_data_to_joints_map.append(f'{f},{id},{orig_id}')    
            continue
        data[...,0] *= -1
        data_m = data.copy()
        data_m[:,l_joints] = data[:,r_joints]
        data_m[:,r_joints] = data[:,l_joints]
        data = np.delete(data,joints_to_drop,axis=1)
        data_m = np.delete(data_m,joints_to_drop,axis=1)
        np.save(out_f,data)
        np.save(out_m_f,data_m)
        pose_data_to_joints_map.append(f'{f},{id},{orig_id}')
        pbar.set_postfix({'samples':2*len(pose_data_to_joints_map),
                          'dropped':n_dropped})
_ = open('pose_data_to_joints_map.txt','w').write('\n'.join(pose_data_to_joints_map) + '\n')
pbar.close()

mirroring/pruning files:   0%|                                                                                              | 0/17138 [00:00<?, ?it/s]

mirroring/pruning files: 100%|█████████████████████████████████████████████████████| 17138/17138 [08:35<00:00, 33.26it/s, samples=26504, dropped=4525]


Checkout original train/val/test sets and texts. Just to make sure we have the original versions.

In [38]:
!git checkout main -- HumanML3D/train.txt HumanML3D/val.txt HumanML3D/test.txt HumanML3D/texts.zip
!rm -rf HumanML3D/train_orig.txt HumanML3D/val_orig.txt HumanML3D/test_orig.txt HumanML3D/texts_orig HumanML3D/texts

In [39]:
import shutil

_ = shutil.move('HumanML3D/train.txt','HumanML3D/train_orig.txt')
_ = shutil.move('HumanML3D/val.txt','HumanML3D/val_orig.txt')
_ = shutil.move('HumanML3D/test.txt','HumanML3D/test_orig.txt')

Make new test set from original test set with missing samples removed. SMPL-X version of AMASS doesn't seem to have all the original samples from SMPL-H version.

In [40]:
orig_test = [l.strip()
             for l in open('./HumanML3D/test_orig.txt')]
print('orig test set:',len(orig_test))

pose_data_to_joints_map_f = pathlib.Path('pose_data_to_joints_map.txt')
orig_id_to_new_id = {row['orig_id']:row['new_id']
                     for row in csv.DictReader(open(pose_data_to_joints_map_f))}

index_f = pathlib.Path('./index.csv')
orig_id_to_orig_file = {row['new_name'].replace('.npy',''):row['source_path']
                        for row in csv.DictReader(open(index_f))}

new_test = list()
for orig_id in orig_test:
    is_m_id = orig_id.startswith('M')
    if is_m_id:
        orig_m_id = orig_id
        orig_id = orig_m_id[1:]
    else:
        orig_m_id = f'M{orig_id}'
    data_f = orig_id_to_orig_file[orig_id]
    # Not using `humanact12` dataset. Discard those entries in val set.
    if 'humanact12' in data_f:
        continue
    # SMPL-X version of AMASS is missing the following. Ignore them.
    if ('BMLhandball' in data_f or
        'DanceDB' in data_f or
        'HUMAN4D' in data_f or 
        'CMU/22_23_Rory' in data_f or
        'CMU/18_19_rory' in data_f or
        'CMU/18_19_Justin' in data_f or
        'CMU/20_21_rory1' in data_f or
        'CMU/20_21_Justin1' in data_f or
        'CMU/22_23_justin' in data_f):
        continue
    new_test.append(orig_id_to_new_id[orig_id])
open('HumanML3D/test.txt','w').write('\n'.join(sorted(new_test))+'\n')
print('new test set:',len(new_test))

orig test set: 4384
new test set: 3980


Use the remaining samples as the train set.

In [41]:
orig_train = [l.strip()
             for l in open('./HumanML3D/train_orig.txt')]
print('orig train set:',len(orig_train))

new_test = set(new_test)
data_d = pathlib.Path('./joints')
new_train = list()
for f in tqdm(list(sorted(data_d.iterdir())),desc='sample',ncols=150):
    id = f.with_suffix('').name
    if id not in new_test:
        new_train.append(id)
_ = open('HumanML3D/train.txt','w').write('\n'.join(sorted(new_train)) + '\n')
print('new train set:',len(new_train))

orig train set: 23384


sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 26502/26502 [00:00<00:00, 413800.02it/s]

new train set: 24512


Find the corresponding text file for each example in the new train/test set.

In [42]:
!unzip -q HumanML3D/texts.zip -d HumanML3D

In [43]:
import shutil
import csv

shutil.move('HumanML3D/texts','HumanML3D/texts_orig')

texts_orig_d = pathlib.Path('HumanML3D/texts_orig')
texts_d = pathlib.Path('HumanML3D/texts')
texts_d.mkdir(parents=True,exist_ok=True)

pose_data_to_joints_map_f = pathlib.Path('pose_data_to_joints_map.txt')
new_id_to_orig_id = {row['new_id']:row['orig_id']
                     for row in csv.DictReader(open(pose_data_to_joints_map_f))}
for f in tqdm(list(sorted(data_d.iterdir())),desc='sample',ncols=150):
    id = f.with_suffix('').name
    text_f = texts_d / f'{id}.txt'
    if id.startswith('M'):
        id = id[1:]
    orig_id = new_id_to_orig_id[id]
    text_orig_f = texts_orig_d / f'{orig_id}.txt'
    shutil.copyfile(text_orig_f,text_f)

sample: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 26502/26502 [00:09<00:00, 2710.06it/s]
